In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import timeit

In [2]:
# Boris-Method Particle push
def BorisPush(initialPosition, initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun):

    timeSteps = np.linspace(0, timeofrun, numberOfitteration)
    dt = timeSteps[1] - timeSteps[0]

    position = np.zeros([numberOfitteration, 3])
    velocity = np.zeros([numberOfitteration, 3])
    kinetic_energy = np.zeros(numberOfitteration)

    position[0] = initialPosition
    velocity[0] = initialVelocity

    kinetic_energy[0] = np.linalg.norm(velocity[0])

    t = (chargeofparticle*magneticfield/massofparticle)*dt/2
    s = (2*t)/(1 + np.dot(t,t))

    for i in range(numberOfitteration-1):
        # Kick
        position_half = position[i] + velocity[i]*dt/2

        # Drift and rotation
        v_minus = velocity[i] + (chargeofparticle*electricfield/massofparticle)*dt/2
        v_prime = v_minus + np.cross(v_minus, t)
        v_plus = v_minus + np.cross(v_prime, s)
        velocity[i+1] = v_plus + (chargeofparticle*electricfield/massofparticle)*dt/2

        # Kick
        position[i+1] = position_half + velocity[i+1]*dt/2
        kinetic_energy[i+1] = np.linalg.norm(velocity[i+1])

    return position, velocity, kinetic_energy, timeSteps


In [3]:
# EulerMethod for Particle push
def EulerMethod(initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun):
    timeSteps = np.linspace(0, timeofrun, numberOfitteration)
    dt = timeSteps[1] - timeSteps[0]

    position = np.zeros([numberOfitteration, 3])
    velocity = np.zeros([numberOfitteration, 3])
    kinetic_energy = np.zeros(numberOfitteration)

    velocity[0] = initialVelocity
    kinetic_energy[0] = np.linalg.norm(velocity[0])

    for i in range(numberOfitteration-1):
        velocity[i+1] = velocity[i] + (chargeofparticle/massofparticle)*(
            electricfield + np.cross(velocity[i], magneticfield))*dt
        position[i+1] = position[i] + velocity[i]*dt
        kinetic_energy[i+1] = np.linalg.norm(velocity[i+1])
    return position, velocity, kinetic_energy, timeSteps


In [4]:
# Runge-Kutta-Method (4th order)
def RungeKuttaMethod(initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun):
    timeSteps = np.linspace(0, timeofrun, numberOfitteration)
    dt = timeSteps[1] - timeSteps[0]

    position = np.zeros([numberOfitteration, 3])
    velocity = np.zeros([numberOfitteration, 3])
    kinetic_energy = np.zeros(numberOfitteration)

    velocity[0] = initialVelocity
    kinetic_energy[0] = np.linalg.norm(velocity[0])

    for i in range(numberOfitteration-1):
        v1 = (chargeofparticle/massofparticle) * \
            (electricfield + np.cross(velocity[i], magneticfield))*dt
        v2 = (chargeofparticle/massofparticle)*(electricfield +
                                                np.cross(velocity[i]+v1/2, magneticfield))*dt
        v3 = (chargeofparticle/massofparticle)*(electricfield +
                                                np.cross(velocity[i]+v2/2, magneticfield))*dt
        v4 = (chargeofparticle/massofparticle) * \
            (electricfield + np.cross(velocity[i]+v3, magneticfield))*dt
        velocity[i+1] = velocity[i] + (1/6)*(v1 + 2*v2 + 2*v3 + v4)

        p1 = velocity[i]*dt
        p2 = (velocity[i] + p1/2)*dt
        p3 = (velocity[i] + p2/2)*dt
        p4 = (velocity[i] + p3)*dt
        position[i+1] = position[i] + (1/6)*(p1 + 2*p2 + 2*p3 + p4)
        kinetic_energy[i+1] = np.linalg.norm(velocity[i+1])

    return position, velocity, kinetic_energy, timeSteps


In [5]:
# Leap-Frog for Particle push
def LeapFrog(initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun):
    timeSteps = np.linspace(0, timeofrun, numberOfitteration)
    dt = timeSteps[1] - timeSteps[0]

    position = np.zeros([numberOfitteration, 3])
    velocity = np.zeros([numberOfitteration, 3])
    kinetic_energy = np.zeros(numberOfitteration)

    velocity[0] = initialVelocity
    kinetic_energy[0] = np.linalg.norm(velocity[0])

    def residual(velocity, v_half, dt):
        return velocity - v_half - 0.5 * (chargeofparticle/massofparticle)*(electricfield + np.cross(velocity, magneticfield))*dt

    for i in range(numberOfitteration-1):
        v_half = velocity[i] + 0.5 * (chargeofparticle/massofparticle)*(
            electricfield + np.cross(velocity[i], magneticfield))*dt
        position[i+1] = position[i] + v_half * dt

        # sing Fsolve to get the roots of the equation
        velocity[i + 1] = fsolve(residual, v_half, args=(v_half, dt))
        kinetic_energy[i+1] = np.linalg.norm(velocity[i+1])

    return position, velocity, kinetic_energy, timeSteps

In [6]:
# Initial Conditions and parameters to initialize the  particle pusher
numberOfitteration = 500
timeofrun = 10
electricfield = np.array([0, 0, 0])
magneticfield = np.array([0, 0, 1])
massofparticle = 0.1
chargeofparticle = 1
initialVelocity = np.array([0, 1, 0])

time1 = timeit.default_timer()
position_B, velocity_B, kinetic_energy_B, timeSteps_B = BorisPush(
    initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun)
time2 = timeit.default_timer()
print(f'Time taken to run Boris push for {numberOfitteration} iteration is: {time2-time1}')
# position_E, velocity_E, kinetic_energy_E, timeSteps_E = RungeKuttaMethod(
#     initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun)
# position_R, velocity_R, kinetic_energy_R, timeSteps_R = EulerMethod(
#     initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun)
time3=timeit.default_timer()
position_L, velocity_L, kinetic_energy_L, timeSteps_L = LeapFrog(
    initialVelocity, electricfield, magneticfield, chargeofparticle, massofparticle, numberOfitteration, timeofrun)
time4 = timeit.default_timer()
print(f'Time taken to run LeapFrog push for {numberOfitteration} iteration is: {time4-time3}')

from mpl_toolkits import mplot3d
%matplotlib qt
plt.axes(projection= '3d')
plt.scatter(position_B[:,0], position_B[:,1], timeSteps_B, c='red');
# plt.xlabel('x')
# plt.ylabel('y')
# plt.zl('time')
# plt.show()

from mpl_toolkits import mplot3d
%matplotlib qt
plt.axes(projection= '3d')
plt.plot(position_B[:,0], position_B[:,1], timeSteps_B);
plt.xlabel('x')
plt.ylabel('y')
# plt.zl('time')
plt.show()


# plt.axes(projection= '3d')
# plt.plot(position_L[:,0], position_L[:,1], timeSteps_L, c="blue");
# plt.xlabel('x')
# plt.ylabel('y')
# plt.zl('time')
plt.show()

TypeError: BorisPush() missing 1 required positional argument: 'timeofrun'

In [ ]:
x = position_B[:,0]
x
x_1 =np.argmin(x-0)
x_1

377

In [ ]:
# # Initializing conditions
# N = 10
# t = np.linspace(0, 10, N)
# dt = t[1] - t[0]

# E = np.array([0, 0, 0])
# B = np.array([0, 0, 1])
# m = 1
# q = 1
# v_in = np.array([1, 0, 0])


# # For Boris-method
# position_B = np.zeros([N, 3])
# velocity_B = np.zeros([N, 3])
# kinetic_energy_B = np.zeros(N)

# velocity_B[0] = v_in
# kinetic_energy_B[0] = np.linalg.norm(velocity_B[0])

# # For Euler-method
# position_E = np.zeros([N, 3])
# velocity_E = np.zeros([N, 3])
# kinetic_energy_E = np.zeros(N)

# velocity_E[0] = v_in
# kinetic_energy_E[0] = np.linalg.norm(velocity_E[0])

# # For RK-4 method
# position_RK4 = np.zeros([N, 3])
# velocity_RK4 = np.zeros([N, 3])
# kinetic_energy_RK4 = np.zeros(N)

# velocity_RK4[0] = v_in
# kinetic_energy_RK4[0] = np.linalg.norm(velocity_RK4[0])


# for i in range(N-1):

#     # boris-method
#     velocity_B[i+1] = borisPush(velocity_B[i], E, B, q, m, dt)
#     position_B[i+1] = position_B[i] + velocity_B[i+1]*dt
#     kinetic_energy_B[i+1] = np.linalg.norm(velocity_B[i+1])

#     # Euler-method
#     velocity_E[i+1] = eulerPush(velocity_E[i], E, B, q, m, dt)
#     position_E[i+1] = position_E[i] + velocity_E[i+1]*dt
#     kinetic_energy_E[i+1] = np.linalg.norm(velocity_E[i+1])

#     # RK-4 Method
#     velocity_RK4[i+1] = rungeKutta4Method(velocity_RK4[i], E, B, q, m, dt)
#     position_RK4[i+1] = position_RK4[i] + velocity_RK4[i+1]*dt
#     kinetic_energy_RK4[i+1] = np.linalg.norm(velocity_RK4[i+1])


In [ ]:
from mpl_toolkits import mplot3d
%matplotlib widget
plt.axes(projection= '3d')
plt.plot(position_B[:,0], position_B[:,1], timeSteps_B);
plt.xlabel('x')
plt.ylabel('y')
# plt.zl('time')
plt.show()

In [ ]:
# plt.figure(figsize=(10, 6), dpi=100)
# plt.plot(timeSteps_B, kinetic_energy_B, label='BorisPush')
# plt.plot(timeSteps_L, kinetic_energy_L, label='LeapFrog')
# # plt.plot(timeSteps_R, kinetic_energy_R, label='RungeKuttaMethod')
# # plt.plot(timeSteps_E, kinetic_energy_E, label='EulerMethod')
# plt.legend()


In [ ]:
plt.axes(projection= '3d')
plt.plot(velocity_B[:,0], velocity_B[:,1], timeSteps_B);
plt.xlabel('v_x')
plt.ylabel('v_y')
plt.show()

In [ ]:
for i in range(numberOfitteration):
    if (position_B[i,0] < 0.01 and position_B[i,0] > -0.01):
        print(position_B[i,1])

0.0


In [ ]:
def closest(lst, K):
      
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

In [ ]:
for i in range(numberOfitteration):
    if (position_B[i, 0] < 0.0000030 and position_B[i, 0] > -0.0000030):
        plt.scatter(i, position_B[i+2, 1]-position_B[i, 1])
        print(i)
        print(position_B[i, 1]-position_B[i+1, 1])
plt.show()

ValueError: s must be a scalar, or float array-like with the same size as x and y

In [ ]:
t = (chargeofparticle*magneticfield/massofparticle)
t

array([ 0.,  0., 10.])

In [ ]:
t**2

array([  0.,   0., 100.])